https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

In [165]:
import csv

In [178]:
def loadCsv(filename):
    lines = csv.reader(open(filename, mode = "r"))
    dataset = list(lines)
    
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]] # float[x] would have converted all values to floats
    return dataset

In [141]:
#filename = 'exampleinput.csv'
#datain = loadCsv(filename)

In [142]:
#print('Loaded data file {0} with {1} rows'.format(filename, len(datain)))

In [143]:
#datain.pop(0) # removes header - this was done when data had a header row
# Note everytime you pop, it removes the first line!!!

In [144]:
#dataset = datain #this was used to remove the top row

#### Splitting the dataset

In [145]:
import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return[trainSet, copy]

In [146]:
#splitRatio = 0.67
#train, test = splitDataset(dataset, splitRatio)
#print('Training data:', train, '\n\n\nTest data:', test)

In [147]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [148]:
separated = separateByClass(dataset)
#print('Separated instances:', separated) # separates between those of class where R=1, and where R=0,
# it assumes that the last value is the class (last column R)

In [149]:
#### Calculate mean and standard deviation of each of the attributes for a class value

In [179]:
import math
def mean(numbers):
    return ((sum(numbers))/float(len(numbers)))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/(len(numbers)-1)
    return math.sqrt(variance)

In [151]:
#numbers = [1,2,3,4,5,6,7,8,9]
#print('Summary of:', numbers, '\nmean:', mean(numbers), '\nstandard deviation:',  stdev(numbers))

In [152]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [153]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [154]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [155]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [156]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [157]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions
 
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0
 

In [183]:
def main():
    filename = 'exampleinput.csv'
    splitRatio = 0.50
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split', len(dataset), 'into Test', len(trainingSet),'rows', 'and', 
          len(testSet), 'Train rows')
    # prepare model
    summaries = summarizeByClass(trainingSet)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy:', accuracy, '%')
 
main()

Split 149 into Test 74 rows and 75 Train rows
Accuracy: 69.33333333333334 %
